In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from shapely.geometry import Point

In [ ]:
response=requests.get('https://en.wikipedia.org/wiki/List_of_deadly_earthquakes_since_1900')

In [ ]:
result_text=response.text

In [ ]:
print(type(response))
print(type(result_text))

In [ ]:
result_text

In [ ]:
soup = BS(result_text, 'html.parser')

In [ ]:
# looks for wikitable first table does not contain valuable information
table = soup.find_all('table',{'class':'sortable wikitable'})

In [ ]:
type(table)

In [ ]:
df=pd.read_html(str(table)) #converts html table into dataframe

In [ ]:
df

In [ ]:
df=df[0] #selects 1st table (I think this happens to be the only table)

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# converts all missing values to NaN
df1=df.replace(r'\s+$', np.nan, regex=True)

In [ ]:
df1.describe()

In [ ]:
df1.info()

In [ ]:
df1
# As you note, all missing values are filled in with NaN already.  No visible change between df and df1.  
# will continue to use df

In [ ]:
df == df1 #Why are there false values

In [ ]:
df.iloc[1340,-1]

In [ ]:
# Need to remove all "footnotes" from values in column

In [ ]:
type(df.iloc[1340,-1])

In [ ]:
OSDsep=df.iloc[:,11].str.partition('[')

In [ ]:
OSDsep.head()

In [ ]:
OSDsep1=OSDsep.iloc[:,0]

In [ ]:
print(OSDsep1.head())

In [ ]:
OSDsep2=OSDsep1.str.partition('(')

In [ ]:
OSDsep2.tail()

In [ ]:
OSDsep3=OSDsep2[0].str.partition('+')

In [ ]:
OSDsep3.tail()

In [ ]:
OSDsep4=OSDsep3[0]

In [ ]:
OSDsep4.describe()

In [ ]:
type(OSDsep4)

In [ ]:
# df.iloc[:,11].replace(to_replace=OSDsep4)
df.iloc[:,11]=OSDsep4

In [ ]:
df.tail()

In [ ]:
type(df.iloc[1335,-1])# went to previous steps to change str to float 

In [ ]:
df.iloc[:,11]=pd.to_numeric(df.iloc[:,11],errors='coerce')

In [ ]:
type(df.iloc[1340,-1])

In [ ]:
df.head(10)

In [ ]:
df.tail()

In [ ]:
df.columns

In [ ]:
df.iloc[0,11]='Other Source Deaths'

In [ ]:
df.iloc[0,11]

In [ ]:
df.columns=df.iloc[0,:]

In [ ]:
df.columns

In [ ]:
df.head()#should I delete row 0 now that I've changed the column names to reflect what was in row 0

In [ ]:
mag=df.iloc[:,5].str.partition(' ')

In [ ]:
mag.head(3)

In [ ]:
df.iloc[:,5]=mag[0] #Assume magnitude variables (i.e. Muk, Mk, etc are neglible regardless
# of differing scales)
df.head()

In [ ]:
df.columns

In [ ]:
type(df.iloc[4,5])

In [ ]:
# df.iloc[:,2]=pd.to_numeric(df.iloc[:,11],errors='coerce')
# df.iloc[:,3]=pd.to_numeric(df.iloc[:,11],errors='coerce')
# df.iloc[:,4]=pd.to_numeric(df.iloc[:,11],errors='coerce')
df.iloc[:,5]=pd.to_numeric(df.iloc[:,5],errors='coerce')
# df.iloc[:,6]=pd.to_numeric(df.iloc[:,6],errors='coerce')
df.iloc[:,7]=pd.to_numeric(df.iloc[:,7],errors='coerce')
df.iloc[:,8]=pd.to_numeric(df.iloc[:,8],errors='coerce')
df.iloc[:,9]=pd.to_numeric(df.iloc[:,9],errors='coerce')
df.iloc[:,10]=pd.to_numeric(df.iloc[:,10],errors='coerce')
df.iloc[:,11]=pd.to_numeric(df.iloc[:,11],errors='coerce')

In [ ]:
df.head(3)

In [ ]:
type(df.iloc[28,6])

In [ ]:
df['deaths']=df.iloc[:,8:].apply(np.max,axis=1)
# df.rename(mapper={"Present-day country and link to Wikipedia article":"Country"})
df.columns =['Origin(UTC)','Country','Lat','Long','Depth(km)','Magnitude','Secondary Effects', 'PDE Shaking Deaths', 'PDE Total Deaths', 'Utsu Total Deaths', 'EM-DAT Total Deaths', 'Other Source Deaths','deaths']   
df['Country'].str.replace(r" ?\([^)]+\)", "",regex=True).unique()

df

In [ ]:
fix2=df['Country'].str.partition('(')
df['Country']=fix2.iloc[:,0]
df.tail()

In [ ]:
# df['deaths']= df.iloc[:,8:].max()
# df.agg("max", axis="columns")
# list1=df['PDE Total Deaths']
# list2=df['Utsu Total Deaths']
# list3=df['EM-DAT Total Deaths']
# list4=df['Other Source Deaths']

# def function(list1,list2,list3,list4): #def returns 5th list 
#     df['deaths'] = [max(value) for value in zip(list1, list2, list3, list4)]
#     return df['deaths']
# print(function(list1,list2,list3,list4))

Data Question3
- Nate Silver discusses the difficulty of predicting earthquakes in **The Signal and the Noise**, but are there factors that make an earthquake more likely? 
- Are there factors that make an earthquake more deadly? 
- Where would you live if you wanted to eliminate the risk of experiencing an earthquake? 
- Where would you avoid living?

Read the table of earthquakes from https://en.wikipedia.org/wiki/List_of_deadly_earthquakes_since_1900 using *beautifulsoup* and load it to a pandas dataframe. An introduction to the request and beautifulsoup libraries is provided in `TuringAward_soup.ipynb`.

Consider
 - Define "Deadly"

In [ ]:
sns.residplot(x=df.Magnitude,y=df.deaths,data=df.deaths)

In [ ]:
# sns.stripplot(x=df.Country,y=df.Magnitude,data=df.Magnitude,jitter=True)
# sns.jointplot(x=df.Country, y=df.Magnitude, data=df.Magnitude,kind='scatter')
# plt.plot(df.Country,df.Magnitude)
# mag=df.Magnitude
# death=df.deaths
# place=df.Country
# # plt.plot(place,mag)
# df.plot(kind='bar',x=Country,y=Magnitude)
# plt.show()

In [ ]:
Max_death=df['deaths'].max()
maxdeath=df.loc[df['deaths']==Max_death]
print(maxdeath)
#Why is this value so much higher than all the other death values?  Perhaps there's a greater population density?
#building structures?

Consider reviewing average deaths by country over the years....  Where does China fall in to that with this ID#584 
 and without....

In [ ]:
# rolling_df_max=rolling_df.groupby('User').max().reset_index()
# rolling_df_max.head()
Country=df.groupby('Country').mean()
print(Country)

In [ ]:
type(Country)

In [ ]:
Country.head()

In [ ]:
country_count=df['Country'].value_counts()
#How many earthquakes are too many... normalize points and find percentage of earthquakes...
# SumofQ=country_count.sum() --- Duh... total number of rows in df = 1341
country_count=pd.DataFrame(country_count).reset_index()
country_count.columns=['Country', 'Number of Earthquakes']
country_count.head()

In [ ]:
country_count['percentage'] =(country_count['Number of Earthquakes']/1341)*100
country_count.head(50)

In [ ]:
country_count.plot(kind='bar',x='Country', y='Number of Earthquakes', title='Number of Earthquake Occurrences per Country since 1900')
plt.show()

In [ ]:
topEQs=country_count.loc[country_count['Number of Earthquakes']>=60]  #presumably 1 every 2-3 years - This parameter
# can be changed

In [ ]:
topEQs.plot(kind='bar',x='Country', y='Number of Earthquakes', title='Number of Earthquake Occurrences since 1900 for High "Output" Countries')
plt.show()

In [ ]:
plt.pie(topEQs['percentage'],labels=topEQs['Country'])
plt.title='Percentage of Earthquakes for High "Output" Countries' #Fix title later...
plt.show()

Need to find and compare countries with high death toll

Find tectonic activity and compare to location of earthquakes

In [ ]:
#Drop NaN values
# dfnona=df.loc[df['Lat'].notna()]
df.tail()

In [ ]:
df.head()

In [ ]:
df.iloc[:,2]=pd.to_numeric(df.iloc[:,2],errors='coerce')
df.iloc[:,3]=pd.to_numeric(df.iloc[:,3],errors='coerce')
df.iloc[:,4]=pd.to_numeric(df.iloc[:,4],errors='coerce')
df.tail()

In [ ]:
dfmap=df.loc[df['Lat'].notna()]
dfmap.shape

In [ ]:
dfmap2=dfmap.loc[dfmap['Long'].notna()]
dfmap2.shape

GeoJSON Data found @ https://www.gislounge.com/find-tectonic-plate-gis-data/ and https://github.com/fraxen/tectonicplates

In [ ]:
plates=gpd.read_file('C:/Users/unews/nss_data_science/data-question-3-earthquakes-the-dream-crushers/tectonicplates/GeoJSON/PB2002_plates.json')

In [ ]:
plates.loc[0,'geometry']

The following cells combine the location of earthquakes juxtaposed with the location of tectonic plates using GeoJSON.

In [ ]:
plates_kwds = {'title': 'Plates', 'loc': 'upper left', 'bbox_to_anchor': (1, 1.03), 'ncol': 6}
plates.plot(column = 'PlateName', figsize=(12, 12), legend = True, legend_kwds = plates_kwds)
plt.show()

Location of earthquakes on the world map.  Circles are indicative of location and diameter represents magnitude of earthquake

In [ ]:
world=folium.Map(location=[20,0], tiles="Mapbox Bright", zoom_start=2)
for i in range(0,len(dfmap2)):
#     folium.Marker([dfmap2.iloc[i]['Lat'], dfmap2.iloc[i]['Long']], popup=dfmap2.iloc[i]['Country']).add_to(world)
    folium.Circle(
        location=[dfmap2.iloc[i]['Lat'],dfmap2.iloc[i]['Long']],
        popup=dfmap2.iloc[i]['Country'],
        radius=dfmap2.iloc[i]['Magnitude']*10000,
        color='crimson',
        fill=True,
        fill_color='crimson').add_to(world)
world

# Make an empty map
# m = folium.Map(location=[20,0], tiles="Mapbox Bright", zoom_start=2)
 
# # I can add marker one by one on the map
# for i in range(0,len(data)):
#    folium.Circle(
#       location=[data.iloc[i]['lon'], data.iloc[i]['lat']],
#       popup=data.iloc[i]['name'],
#       radius=data.iloc[i]['value']*10000,
#       color='crimson',
#       fill=True,
#       fill_color='crimson'
#    ).add_to(m)
 
# Save it as html
# m.save('mymap.html')

This should help to answer the question... "Where to and where not to live..."

In [ ]:
world=folium.Map(location=[20,0], tiles="Mapbox Bright", zoom_start=2)
folium.GeoJson(plates.geometry).add_to(world)
for i in range(0,len(dfmap2)):
#     folium.Marker([dfmap2.iloc[i]['Lat'], dfmap2.iloc[i]['Long']], popup=dfmap2.iloc[i]['Country']).add_to(world)
    folium.Circle(
        location=[dfmap2.iloc[i]['Lat'],dfmap2.iloc[i]['Long']],
        popup=(str(dfmap2.iloc[i]['Country']) + '<br/>' +
       'EventTime: ' + str(dfmap2.iloc[i]['Origin(UTC)']) + '<br/>' +
       'Magnitude: ' + str(dfmap2.iloc[i]['Magnitude']) + '<br/>' +
       'Deaths: ' + str(dfmap2.iloc[i]['deaths'])).replace("'", "`"),
        radius=dfmap2.iloc[i]['Magnitude']*10000,
        color='crimson',
        fill=True,
        fill_color='crimson').add_to(world)
world

What's the relationship between secondary effects and number of deaths?

In [ ]:
seceff=df.loc[df['Secondary Effects'].notna()]
seceff.shape
# https://clevertap.com/blog/exploring-the-relationship-between-variables-visually/

In [ ]:
# plt.subplot(1,2,1)
seceff.boxplot(column='deaths',by='Secondary Effects')
plt.ylim([0,150000]) #negates large value of 300,000 deaths
# plt.subplot(1,2,2)
# seceff.plot(x='Secondary Effects', y='deaths', kind='box')
plt.show()


In [ ]:
# plt.subplot(1,2,1)
seceff.boxplot(column='deaths',by='Secondary Effects')
plt.ylim([0,20000]) #negates larger values
# plt.subplot(1,2,2)
# seceff.plot(x='Secondary Effects', y='deaths', kind='box')
plt.show()

In [ ]:
# plt.subplot(1,2,1)
seceff.boxplot(column='deaths',by='Secondary Effects')
plt.ylim([0,12500]) #negates larger values
# plt.subplot(1,2,2)
# seceff.plot(x='Secondary Effects', y='deaths', kind='box')
plt.show()

In [ ]:
# plt.subplot(1,2,1)
seceff.boxplot(column='deaths',by='Secondary Effects')
plt.ylim([0,5000]) #negates larger values
# plt.subplot(1,2,2)
# seceff.plot(x='Secondary Effects', y='deaths', kind='box')
plt.show()

In [ ]:
seceff.plot(y='deaths', kind='box')
plt.show()

In [ ]:
SE=seceff['Secondary Effects']
deatheff=seceff['deaths']
plt.subplot(1,2,2)
sns.stripplot(x = SE, y = deatheff, size=4,jitter=True)
# plt.ylim(0,5000)
plt.subplot(1,2,1)
sns.boxplot(x=seceff['Secondary Effects'], y=seceff['deaths'])
plt.ylim(0,5000)
# plt.subplot(2,2,1)
# sns.stripplot(x = SE, y = deatheff, size=4,jitter=True)
# plt.ylim([0,150000])
# plt.subplot(2,2,2)
# sns.boxplot(x=seceff['Secondary Effects'], y=seceff['deaths'])
# plt.ylim([0,150,000])
plt.show()

"There are two main reasons to use logarithmic scales in charts and graphs. The first is to respond to skewness towards large values; i.e., cases in which one or a few points are much larger than the bulk of the data." https://www.forbes.com/sites/naomirobbins/2012/01/19/when-should-i-use-logarithmic-scales-in-my-charts-and-graphs/#71d65d275e67

In [ ]:
#plotting on logarithmic scale
seceff.boxplot(column='deaths',by='Secondary Effects',figsize=(20,10))
plt.yscale('log')
plt.show()

According to wikipedia, T, L, F, Lq represents, "tsunami, landslide, fire, liquefaction "

It appears that earthquakes with the secondary effects of a tsunami and fire concurrently 
appeared to relatively cause more fatalities.  In addition when looking at 
occurrences of a single reported phenomenon such as fire OR Landslides OR 
tsunamis,etc., fires seemed to have the most affect on fatalities followed by liquefaction.  What factors relate to fire or liquefaction?    Perhaps this is 
because fires may have occurred more often compared to other secondary effects.

Complete a value count to determine...

In [ ]:
SE3=SE2[['Origin(UTC)', 'Country', 'Magnitude', 'Secondary Effects', 'deaths']]

In [ ]:
# Attempts to change the name for the graph... not necessary now, but 
# may change later.

# for i in range(0,len(SE2)):
#     SE2.iloc[i]['Secondary Effects']={'T':'Tsunami', 'L':'Landslide', 'F':'Fire','Lq':'Liquefaction'}

# SE3=SE2[SE2['Secondary Effects'].replace(to_replace=('T','L','F','Lq'),value=('Tsunami', 'Landslide', 'Fire','Liquefaction'))]
# SE3=SE3[SE2['Secondary Effects'].replace(to_replace={'T':'Tsunami', 'L':'Landslide', 'F':'Fire','Lq':'Liquefaction})]

In [ ]:
SE3['Secondary Effects'].value_counts()

This answers my question.... Fire and Tsunami/Fires do not occur more often.  Landslides occur more often.  Nevertheless, despite the fact that they don't occur more often, it appears to play a greater effect on fatalities.

Why are fires and tsunami/fires more deadly?  Where were they the deadliest?

Look at where liquefaction, landslides, fires, tsunami/fires occur.....compare to total population if known during that time.

In [ ]:
Lq_countries=SE3.loc[SE3['Secondary Effects']=='Lq']
Lq_countries.shape

In [ ]:
Lq_countries['Country'].value_counts()

In [ ]:
Lq_countries.head()

In [ ]:
# MaxdeathLq=Lq_countries.loc[Lq_countries['deaths']==Lq_countries['deaths'].max()]
# print(MaxdeathLq)

In [ ]:
# Trying to figure out a function that creates the maximum value for death
# and the country for that particular secondary effect


# def maxdeath()
#     for SE in SE2['Secondary Effects']
#     maxdeath='SE'_countries.loc


# print__________
# return _________-

In [ ]:
#Looking at landslides
L_countries=SE3.loc[SE3['Secondary Effects']=='L']
L_countries['Country'].value_counts()

Top 5 Landslide countries are 
Iran                  10
Taiwan                10
Indonesia              8
China                  8
Afghanistan            8

These countries have had the most landslides, but how many deaths have occured? ...
consider using groupby to find the death tolls

In [ ]:
L_max_countries=L_countries.sort_values(['deaths'],ascending=False)
tp=0.1*(len(L_countries))
top=round(tp)
L_max_countries.head(top)

This describes the top 10% of occurrences that produced the most deaths for landslides and their countries.

In [ ]:
F_countries=SE3.loc[SE3['Secondary Effects']=='F']
F_countries['Country'].value_counts()
# Why is Japan listed twice?  Perhaps there's a space somewhere in the string...

In [ ]:
F_max_countries=F_countries.sort_values(by='deaths',ascending=False)
# tp=0.1*(len(F_countries))
# top=round(tp)
# F_max_countries.head(top)
F_max_countries

In [ ]:
max_death_fire=F_countries.loc[F_countries['deaths']==F_countries['deaths'].max()]
print(max_death_fire)

Japan had the deadliest fire related earthquake in 1948.  Out of 5 of the highest fatal occurences due to fire, three of them were in Japan.  All of them occurred before 1950, however, there have been deadly earthquakes to occur in Japan since then.  They may not have had a secondary effect associated with it... Perhaps, Japan should be investigated independently.

The countries with a fire produced as a secondary effect and the number of occurences per country: 
Indonesia         2
Japan             2
Nicaragua         1
China             1
El Salvador       1
Japan             1
United States     1

*Note the number of occurrences are fewer (only 9), yet they seem to produce more fatalities than Tsunamis which have more occurrences.

In [ ]:
T_countries=SE3.loc[SE3['Secondary Effects']=='T']
T_countries['Country'].value_counts()

In [ ]:
T_max_countries=T_countries.sort_values(by='deaths',ascending=False)
tp=0.1*(len(T_countries))
top=round(tp)
T_max_countries.head(top)
#Looking for countries that were affected by specific instances.

In countries plagued by Tsunami-related earthquakes, the top 5 included: Indonesia with 19 occurrences, Japan with 19, Mexico with 12, Chile with 11, and Peru with 8.

Obviously, these countries lie in coastal regions, nevertheless Italy, Guatemala, Chile, and Japan had the highest deaths reported for a Tsunami-related earthquake.

Aside, from Italy, countries with the highest deaths and most Tsunami occurrences border the Pacific Ocean.

While there is a plate that divides the Atlantic Ocean (*find name of plate), tectonic plates (*find name) exist closer in distance to the countries listed along the Pacific Ocean thereby possibly producing a greater effect of Tsunami - related deaths.

# Find population of countries over time and merge with data sets to determing percentage of deaths that occur in country... be mindful of time in year when value is collected.  Could be before or after significant earthquake has occurred.